In [160]:
import pandas as pd
from collections import Counter
import re

df = pd.read_csv('outputNEW2.csv', encoding='utf-8')

In [161]:
df

,Категория,Уровень рейтинга,pr_txt
0,A,A,Повышение кредитного рейтинга Акционерного об...
1,BB,BB,"""«Эксперт РА» подтвердил кредитный рейтинг ком..."
2,A,A,"""НКР повысило кредитный рейтинг ООО """"ОТЭКО-По..."
3,AAA,AAA,"""«Эксперт РА» присвоил кредитный рейтинг ПАО «..."
4,BBB,BBB,"""29 марта 2023 г. Ведущий рейтинговый аналитик..."
...,...,...,...
1195,AAA,AAA,"""«Эксперт РА» подтвердил рейтинг компании «Тат..."
1196,BB,BB,"""«Эксперт РА» понизил рейтинг компании «Инвест..."
1197,A,A+,АКРА подтвердило ПАО «КАМАЗ» кредитный рейтинг...
1198,AAA,AAA,АКРА подтвердило кредитный рейтинг ФГУП «Почта...


In [162]:
pattern = re.compile(r'«[^«»]+»')
pattern2 = re.compile(r'"')
pattern3 = re.compile(r'\b[А-ЯA-Z]+\b')
pattern4 = re.compile(r'<[^<]+>')
pattern5 = re.compile(r'\d+')
pattern6 = re.compile(r'\b[А-ЯA-Z]+\.(?=\s|$)')


df['pr_txt'] = df['pr_txt'].apply(lambda x: pattern.sub('', x))
df['pr_txt'] = df['pr_txt'].apply(lambda x: pattern2.sub('', x))
def remove_all_caps(text):
    return pattern3.sub('', text)

def remove_angle_brackets(text):
    return pattern4.sub('', text)


def remove_numbers(text):
    return pattern5.sub('', text)


# Функция для удаления слов после которых стоит точка
def remove_words_with_period(text):
    text = pattern6.sub('', text) 
    text = text.replace(',', '')  
    text = text.replace('.', '')  
    text = text.replace('—', '')  
    text = text.replace('–', '')  
    text = text.replace('-', '')  
    text = text.replace('(', '')  
    text = text.replace(')', '')  
    text = text.replace('«', '')  
    text = text.replace('»', '')  
    text = text.replace(':', '')  
    text = text.replace(';', '')  
    text = text.replace('/', '')  
    text = text.replace('также', '')  
    text = text.replace('@', '')  

    words = text.split()
    filtered_words = [word for word in words if (not word[0].isupper()) and (len(word) >= 6)] 
    return ' '.join(filtered_words)

def remove_words_starting_with_rating(text):
    # Определяем регулярное выражение для слов, начинающихся на "рейтинг"
    pattern = r'\bрейтинг\w*\b'
    pattern2 = r'\bявляющ\w*\b'
    pattern3 = r'\bкотор\w*\b'
    pattern4 = r'\bкредит\w*\b'
    pattern5 = r'\b=\w*\b'
    
    # Используем регулярное выражение для замены найденных слов на пустую строку
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    text = re.sub(pattern2, '', text, flags=re.IGNORECASE)
    text = re.sub(pattern3, '', text, flags=re.IGNORECASE)
    text = re.sub(pattern4, '', text, flags=re.IGNORECASE)
    text = re.sub(pattern5, '', text, flags=re.IGNORECASE)
    
    return text

def remove_words_with_english_letters(text):
    pattern = r'\b\w*[a-zA-Z]+\w*\b'
    
    text = re.sub(pattern, '', text)
    
    return text

df['pr_txt'] = df['pr_txt'].apply(remove_words_with_english_letters)
df['pr_txt'] = df['pr_txt'].apply(remove_words_starting_with_rating)
df['pr_txt'] = df['pr_txt'].apply(remove_words_with_period)
df['pr_txt'] = df['pr_txt'].apply(remove_angle_brackets)
df['pr_txt'] = df['pr_txt'].apply(remove_all_caps)
df['pr_txt'] = df['pr_txt'].apply(remove_numbers)

TypeError: expected string or bytes-like object

In [163]:
df

,Категория,Уровень рейтинга,pr_txt
0,A,A,Повышение кредитного рейтинга Акционерного об...
1,BB,BB,"""«Эксперт РА» подтвердил кредитный рейтинг ком..."
2,A,A,"""НКР повысило кредитный рейтинг ООО """"ОТЭКО-По..."
3,AAA,AAA,"""«Эксперт РА» присвоил кредитный рейтинг ПАО «..."
4,BBB,BBB,"""29 марта 2023 г. Ведущий рейтинговый аналитик..."
...,...,...,...
1195,AAA,AAA,"""«Эксперт РА» подтвердил рейтинг компании «Тат..."
1196,BB,BB,"""«Эксперт РА» понизил рейтинг компании «Инвест..."
1197,A,A+,АКРА подтвердило ПАО «КАМАЗ» кредитный рейтинг...
1198,AAA,AAA,АКРА подтвердило кредитный рейтинг ФГУП «Почта...


In [145]:
import pymorphy2

# Создаем экземпляр класса MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()

def to_nominative_case(text):
    words = text.split()
    nominative_words = []
    for word in words:
        parsed_word = morph.parse(word)[0]  # Получаем разбор слова
        nominative_form = parsed_word.inflect({'nomn'})  # Приводим к именительному падежу
        if nominative_form:
            nominative_words.append(nominative_form.word)
        else:
            nominative_words.append(word)  # Если не удалось привести, оставляем как есть
    return ' '.join(nominative_words)

# Примените функцию к столбцу 'pr_txt' в DataFrame
df['pr_txt'] = df['pr_txt'].apply(to_nominative_case)

In [146]:
df

,Категория,Уровень рейтинга,pr_txt
0,A,A,общество вызванное улучшение качественная оцен...
1,BB,BB,подтвердивший компания уровень агентство подтв...
2,A,A,повысившее стабильный прогноз влияние стабильн...
3,AAA,AAA,присвоивший уровень агентство присвоившее нефи...
4,BBB,BBB,аналитик директор корпоративные аналитик дирек...
...,...,...,...
1195,AAA,AAA,подтвердивший компания уровень апрель агентств...
1196,BB,BB,понизивший компания уровень изменивший прогноз...
1197,A,A+,подтвердившее прогноз подтвердившее облигации ...
1198,AAA,AAA,подтвердившее уровень прогноз облигации уровен...


In [51]:
top_words_by_rating = {}

# Уникальные уровни рейтинга в данных
unique_ratings = df['Категория'].unique()

# Проходимся по каждому уровню рейтинга
for rating in unique_ratings:
    # Выбираем строки с текущим уровнем рейтинга
    subset = df[df['Категория'] == rating]
    
    # Объединяем все тексты в одну строку
    text = ' '.join(subset['pr_txt'])
    
    # Разделяем текст на слова и фильтруем слова с более чем 3 символами
    words = [word for word in text.split() if len(word) > 4]
    
    # Считаем частоту слов и выбираем топ-20
    word_count = Counter(words)
    top_words = [word for word, count in word_count.most_common(30)]
    
    # Добавляем результат в словарь
    top_words_by_rating[rating] = top_words

# Выводим топ-20 слов (с более чем 3 символами) для каждого уровня рейтинга
for rating, top_words in top_words_by_rating.items():
    print(f'Категория: {rating}')
    print('Топ-30 слов:')
    for i, word in enumerate(top_words, 1):
        print(f'{i}. {word}')
    print()


Уровень рейтинга: A
Топ-30 слов:
1. уровень
2. компания
3. оценка
4. платежи
5. агентство
6. информация
7. присвоение
8. мнение
9. прогноз
10. данные
11. рамки
12. являющийся
13. процентные
14. выручка
15. комитет
16. рентабельность
17. обязательства
18. нагрузка
19. рынок
20. ликвидность
21. являющиеся
22. объём
23. высокая
24. снижение
25. проекты
26. финансовые
27. период
28. итоги
29. отношение
30. чистые

Уровень рейтинга: BB
Топ-30 слов:
1. компания
2. уровень
3. оценка
4. информация
5. агентство
6. прогноз
7. мнение
8. являющийся
9. присвоение
10. данные
11. платежи
12. рентабельность
13. нагрузка
14. рынок
15. ликвидность
16. отчётность
17. управление
18. компании
19. рамки
20. показатель
21. влияние
22. выручка
23. являющиеся
24. финансовые
25. высокая
26. комитет
27. обязательства
28. качество
29. снижение
30. период

Уровень рейтинга: AAA
Топ-30 слов:
1. уровень
2. компания
3. объём
4. агентство
5. группа
6. информация
7. оценка
8. присвоение
9. погашение
10. высокая
11. явл

In [147]:
from collections import Counter

top_words_by_rating = {}

# Уникальные уровни рейтинга в данных
unique_ratings = df['Категория'].unique()

# Создаем общий счетчик слов, которые встречаются во всех рейтингах
all_words_counter = Counter()

# Проходимся по каждому уровню рейтинга
for rating in unique_ratings:
    # Выбираем строки с текущим уровнем рейтинга
    subset = df[df['Категория'] == rating]
    
    # Объединяем все тексты в одну строку
    text = ' '.join(subset['pr_txt'])
    
    # Разделяем текст на слова и фильтруем слова с более чем 3 символами
    words = [word for word in text.split() if len(word) > 4]
    
    # Считаем частоту слов
    word_count = Counter(words)
    
    # Вычитаем из счетчика слов общий счетчик слов, чтобы получить только уникальные слова для текущего рейтинга
    unique_words = word_count - all_words_counter
    
    # Выбираем топ-30 уникальных слов
    top_words = [word for word, count in unique_words.most_common(20)]
    
    # Обновляем общий счетчик слов
    all_words_counter += word_count
    
    # Добавляем результат в словарь
    top_words_by_rating[rating] = top_words

# Выводим топ-30 различных популярных слов для каждого уровня рейтинга
for rating, top_words in top_words_by_rating.items():
    print(f'Категория: {rating}')
    print('Топ-20 различных слов:')
    for i, word in enumerate(top_words, 1):
        print(f'{i}. {word}')
    print()


Категория: A
Топ-20 различных слов:
1. компания
2. уровень
3. оценка
4. информация
5. агентство
6. присвоение
7. мнение
8. платежи
9. являющийся
10. прогноз
11. данные
12. нагрузка
13. высокая
14. рамки
15. комитет
16. выручка
17. являющиеся
18. обязательства
19. рентабельность
20. ликвидность

Категория: BB
Топ-20 различных слов:
1. вагоны
2. растения
3. закупки
4. учебные
5. клиенты
6. пропантовый
7. растениеводство
8. обучение
9. оптовая
10. сельскохозяйственная
11. мясной
12. операторы
13. сельскохозяйственные
14. противотурбулентные
15. посевные
16. отсрочка
17. руководство
18. автошколы
19. нефтедобыча
20. консервация

Категория: AAA
Топ-20 различных слов:
1. государство
2. эмиссия
3. неконвертируемые
4. документарные
5. предъявитель
6. электроэнергия
7. углеводороды
8. корпорация
9. нефтепродукты
10. централизованный
11. системная
12. совместные
13. атомная
14. добыча
15. значимость
16. монополия
17. тарифы
18. паритет
19. передача
20. фактический

Категория: BBB
Топ-20 различны

In [95]:
# Создаем пустой список для хранения данных о категориях и словах
category_words_data = []

# Проходимся по каждой категории и её топ-словам
for category, top_words in top_words_by_rating.items():
    # Соединяем топ-слова через запятую
    words_str = ', '.join(top_words)
    
    # Добавляем информацию о категории и словах в список
    category_words_data.append({'Category': category, 'Words': words_str})

# Записываем данные в файл
with open('category_words.txt', 'w', encoding='utf-8') as f:
    # Проходимся по списку и записываем данные о категории и словах в файл
    for data in category_words_data:
        f.write(f'Category: {data["Category"]}\n')
        f.write(f'Words: {data["Words"]}\n\n')


In [52]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Подготовьте корпус текста
corpus = [word_tokenize(text) for text in df['pr_txt']]

# Обучите модель Word2Vec
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, sg=0)

# Векторизация текста
def text_to_vector(text):
    words = word_tokenize(text)
    vector = sum([model.wv[word] for word in words if word in model.wv])
    return vector

model.save("word2vec_model")

df2 = df
df2['pr_txt'] = df['pr_txt'].apply(text_to_vector)


In [99]:
df

,Категория,Уровень рейтинга,pr_txt
0,A,A,общество далее вызванное улучшение качественна...
1,BB,BB,подтвердивший компания уровень агентство подтв...
2,A,A,повысившее стабильный прогноз влияние стабильн...
3,AAA,AAA,присвоивший уровень агентство присвоившее нефи...
4,BBB,BBB,аналитик директор корпоративные аналитик дирек...
...,...,...,...
1195,AAA,AAA,подтвердивший компания уровень апрель агентств...
1196,BB,BB,понизивший компания уровень изменивший прогноз...
1197,A,A+,подтвердившее прогноз подтвердившее облигации ...
1198,AAA,AAA,подтвердившее уровень прогноз облигации уровен...


In [67]:
print(df2['pr_txt'].iloc[0])

[ 4.3990887e+01  6.0038876e+01 -1.9375337e+02 -3.1942303e+02
  9.6456566e+01 -8.2978653e+01  1.9115045e+02  2.2456201e+02
 -2.2474718e+02 -4.4477182e+02  9.3928940e+01 -7.5776024e+01
 -7.5151415e+00  1.0997385e+02  3.1658472e+02  3.2479172e+01
  4.0676964e+01 -2.8721057e+02 -5.9579948e+01  5.9992462e+01
 -1.6645027e+02  1.2369429e+02  2.9441711e+02  1.6536755e+01
 -1.3376840e+02 -3.8495358e+01  7.6440892e+00 -1.7754736e+02
  2.0086873e+02  1.9162366e+02  3.8454114e+02 -7.2519035e+01
  4.9851128e+01 -4.2192911e+02  7.7482262e+01  2.6132855e+02
  4.1699924e+02  1.4534450e+02 -9.9410896e+01 -1.3965659e+02
  1.0019909e+01 -4.3599515e+02  6.0077389e+01 -4.7933441e+01
  1.2627647e+01  6.2707489e+01  2.7493967e+02  6.8255867e+01
  1.8292865e+02  3.8030907e+01 -2.1241913e+01 -1.8378360e+02
 -1.0221541e+02 -1.0114025e+02 -1.6723079e+02  3.1261655e+02
  2.0271846e+02  6.5283768e+01  1.0185446e+02  3.8169449e+01
  1.4957909e+01  1.9496315e+02  1.5219324e+02  8.0133621e+01
 -1.9160802e+02  3.91529

In [65]:
with open('word_vectors.txt', 'w', encoding='utf-8') as f:
    for word, vector in zip(df['pr_txt'], df2['pr_txt']):
        # Создайте строку, где сначала идет слово, затем его векторное представление
        line = f"{word}: {vector}\n"
        
        # Запишите эту строку в файл
        f.write(line)


In [148]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

# Загрузка данных из CSV файла
data = df

# Разделение данных на текст (X) и целевую метку (y)
X = data['pr_txt']
y = data['Категория']

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Преобразование текста в числовой формат (TF-IDF)
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Вы можете настроить количество признаков
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [103]:
param_grid = {
    'n_estimators': [200],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}


rf = RandomForestClassifier(random_state=42)


grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train_tfidf, y_train)


best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')


best_classifier = grid_search.best_estimator_

y_pred = best_classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.61


In [149]:
from sklearn.linear_model import LogisticRegression


logistic_regression = LogisticRegression(max_iter=1000, random_state=42)


logistic_regression.fit(X_train_tfidf, y_train)


y_pred = logistic_regression.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (Logistic Regression): {accuracy:.2f}')


Accuracy (Logistic Regression): 0.57


In [150]:
from sklearn.svm import SVC


svm = SVC(kernel='linear', C=1, random_state=42)


svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (SVM): {accuracy:.2f}')

Accuracy (SVM): 0.65


In [112]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()


y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


xgboost = xgb.XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)


xgboost.fit(X_train_tfidf, y_train_encoded)


y_pred_encoded = xgboost.predict(X_test_tfidf)


accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f'Accuracy (XGBoost): {accuracy:.2f}')

y_pred = label_encoder.inverse_transform(y_pred_encoded)



Accuracy (XGBoost): 0.62


In [139]:
from sklearn.ensemble import ExtraTreesClassifier

extra_trees = ExtraTreesClassifier(n_estimators=200, random_state=42)


extra_trees.fit(X_train_tfidf, y_train)


y_pred = extra_trees.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (Extra Trees): {accuracy:.2f}')

Accuracy (Extra Trees): 0.66


In [151]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Создайте модель Ridge
ridge = RidgeClassifier(random_state=42)

# Задайте сетку параметров для GridSearchCV
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}

# Создайте объект GridSearchCV
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Обучите модель на обучающих данных
grid_search.fit(X_train_tfidf, y_train)

# Получите наилучшие параметры и модель
best_params = grid_search.best_params_
best_ridge = grid_search.best_estimator_

# Предсказание на тестовом наборе данных
y_pred = best_ridge.predict(X_test_tfidf)

# Оцените производительность модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (Ridge): {accuracy:.2f}')

# Выведите отчет о классификации
classification_rep = classification_report(y_test, y_pred)
print(classification_rep)


Accuracy (Ridge): 0.78
              precision    recall  f1-score   support

           A       0.76      0.87      0.81       109
          AA       0.82      0.85      0.83        59
         AAA       0.98      0.90      0.94        52
           B       0.73      0.42      0.53        19
          BB       0.49      0.51      0.50        39
         BBB       0.80      0.73      0.77        78
           C       1.00      0.75      0.86         4

    accuracy                           0.78       360
   macro avg       0.80      0.72      0.75       360
weighted avg       0.78      0.78      0.78       360



In [152]:

results_df = pd.DataFrame({'pr_txt': X_test, 'Predicted_Category': y_pred})


print(results_df)


                                                 pr_txt Predicted_Category
1178  подтвердивший компания уровень агентство подтв...                 AA
865   подтвердивший компания уровень ноябрь агентств...                  A
101   обусловленный сильная рыночная позиция сильный...                BBB
439   обусловленный сильный бизнеспрофиль высокая ре...                  A
58    обусловленный средние оценки рыночная позиция ...                BBB
...                                                 ...                ...
47    присвоивший уровень установивший развивающийся...                BBB
318   обусловленный сильная рыночная позиция регионы...                 AA
106   подтвердивший уровень декабрь агентство подтве...                AAA
798   обусловленный хорошие рентабельность ликвиднос...                BBB
55    подтвердивший уровень изменивший прогноз разви...                  A

[360 rows x 2 columns]


In [153]:
import joblib

# Сохраните обученную модель в файл
model_filename = 'ridge_model.pkl'
joblib.dump(best_ridge, model_filename)


['ridge_model.pkl']

In [156]:
import joblib

# Загрузите модель из файла
loaded_model = joblib.load('ridge_model.pkl')

# Загрузите текст из файла .txt, который вы хотите классифицировать
with open('bank.txt', 'r', encoding='utf-8') as file:
    input_text = file.read()


# Преобразуйте текст в TF-IDF вектор с использованием того же TfidfVectorizer
input_text_tfidf = tfidf_vectorizer.transform([input_text])

# Выполните предсказание
predicted_category = loaded_model.predict(input_text_tfidf)

print(f'Predicted Category: {predicted_category[0]}')

Predicted Category: C
